# Canny Playground

## Imports, Setup and CUDASIM toggle

In [16]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

import logging

from icecream import ic

from IPython.display import display
import ipywidgets as wid
from utils.ipywidgets_extended import widgets_styling

from utils.setup_notebook import init_notebook
from utils.setup_logging import setup_logging
import utils.memoize as memoize

init_notebook()
setup_logging("INFO")
memoize.set_file_store_path("canny_playground")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
KEY_CUDASIM_ENABLE = "enable_cudasim"
enable_cudasim_checkbox = wid.Checkbox(
    value=memoize.get(KEY_CUDASIM_ENABLE, default=False),
    description="Enable CUDA simulator",
    indent=False,
)
output = wid.Output()

if "cuda_original_setting" not in locals():
    cuda_original_setting = enable_cudasim_checkbox.value

if cuda_original_setting:
    display(wid.HTML("<h1>CUDA  is run via CUDASIM.</h1>"))
    # Enable the CUDA simulator
    os.environ["NUMBA_OPT"] = "0"
    os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
    os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"
else:
    display(wid.HTML("<h1>CUDA is running on the GPU.</h1>"))


@output.capture(clear_output=True, wait=False)
def on_enable_cudasim_checkbox_change(config=None):
    memoize.set(KEY_CUDASIM_ENABLE, enable_cudasim_checkbox.value)

    cuda_sim_setting_changed = enable_cudasim_checkbox.value != cuda_original_setting

    if cuda_sim_setting_changed:
        display(
            wid.HTML(
                "<h1>CUDA simulator setting changed. Please restart the kernel.</h1>"
            )
        )


enable_cudasim_checkbox.observe(on_enable_cudasim_checkbox_change, names="value")
display(wid.VBox([enable_cudasim_checkbox, output]))

HTML(value='<h1>CUDA is running on the GPU.</h1>')

In [18]:
import numpy as np
import pandas as pd
import scipy as sp
import numba as nb
from numba import cuda
import cv2

from utils.benchmarking import LogTimer
from utils.plotting_tools import (
    SmartFigure,
    to_ipy_image,
    plot_kernel,
    plot_matrix,
)
import utils.dyn_module as dyn

from canny_common import load_input_images

logging.getLogger("numba.cuda.cudadrv.driver").setLevel(logging.WARNING)

In [19]:
reset_memoize_store_button = wid.Button(description="Reset memoize store")
reset_memoize_store_button.on_click(lambda x: memoize.reset_store())
display(reset_memoize_store_button)

Button(description='Reset memoize store', style=ButtonStyle())

## Loading Canny Implementations

In [20]:
dir_canny_impls = "./canny_impls"
canny_impls_module_names = dyn.load_modules(dir_canny_impls)

2024-11-05 13:36:46.485 WARNING root Skipping loading of rd_numba_cuda_fp32.py because CUDA is not available
2024-11-05 13:36:46.502 INFO root Loading 1 modules started (..\utils\dyn_module.py:59)
2024-11-05 13:36:46.518 INFO root Reloading rd_vec_v4_dibit started (..\utils\dyn_module.py:26)
2024-11-05 13:36:46.537 INFO root Reloading rd_vec_v4_dibit took: 17.3688 ms (..\utils\dyn_module.py:26)
2024-11-05 13:36:46.553 INFO root Loading 1 modules took: 51.8313 ms (..\utils\dyn_module.py:59)


## Loading Input Images

In [21]:
input_images_dir = "./image_input"
input_images = load_input_images(input_images_dir)

2024-11-05 13:36:46.658 INFO root Loading 6 images started (canny_common.py:18)
2024-11-05 13:36:46.674 INFO root Loading circle_128.png started (..\utils\image_tools.py:64)
2024-11-05 13:36:46.694 INFO root Loading circle_128.png took: 15.9064 ms (..\utils\image_tools.py:64)
2024-11-05 13:36:46.712 INFO root Loading circle_32.png started (..\utils\image_tools.py:64)
2024-11-05 13:36:46.731 INFO root Loading circle_32.png took: 18.9589 ms (..\utils\image_tools.py:64)
2024-11-05 13:36:46.750 INFO root Loading circle_512.png started (..\utils\image_tools.py:64)
2024-11-05 13:36:46.772 INFO root Loading circle_512.png took: 22.3875 ms (..\utils\image_tools.py:64)
2024-11-05 13:36:46.792 INFO root Loading circle_64.png started (..\utils\image_tools.py:64)
2024-11-05 13:36:46.818 INFO root Loading circle_64.png took: 21.5264 ms (..\utils\image_tools.py:64)
2024-11-05 13:36:46.839 INFO root Loading park.png started (..\utils\image_tools.py:64)
2024-11-05 13:36:47.296 INFO root Loading park.p

## Running Canny

In [22]:
fig_width = 16
image_size = 512

KEY_IMAGE_DROPDOWN = "image_dropdown"
image_dropdown_options = [image.filename for image in input_images]
image_dropdown = wid.Dropdown(
    options=image_dropdown_options,
    value=memoize.get(
        KEY_IMAGE_DROPDOWN,
        default=input_images[0].filename,
        possible_values=image_dropdown_options,
    ),
    description="Image",
    **widgets_styling,
)
KEY_CANNY_IMPL_DROPDOWN = "canny_impl_dropdown"
canny_impl_dropdown = wid.Dropdown(
    options=canny_impls_module_names,
    value=memoize.get(
        KEY_CANNY_IMPL_DROPDOWN,
        default=canny_impls_module_names[0],
        possible_values=canny_impls_module_names,
    ),
    description="Canny Implementation",
    **widgets_styling,
)
refresh_button = wid.Button(
    description="Reload Implementation",
    **widgets_styling,
)
output = wid.Output()

# lifted out because the plots must remain open
plot_gauss_smart_fig = SmartFigure()
plot_non_max_smart_fig = SmartFigure()
plot_hysteresis_smart_fig = SmartFigure()


@output.capture(clear_output=True, wait=True)
def on_menu_change(_=None):
    memoize.set(KEY_IMAGE_DROPDOWN, image_dropdown.value)
    memoize.set(KEY_CANNY_IMPL_DROPDOWN, canny_impl_dropdown.value)

    # reload the impl module
    current_canny_impl = canny_impl_dropdown.value
    canny_impl = dyn.load_module(current_canny_impl)

    input_image = input_images[image_dropdown.index]

    # Show the selected image
    image_color = input_image.image_color
    image_gray = input_image.image_gray

    height, width = image_gray.shape

    image_title = f"{image_dropdown.value} ({width}x{height})"
    display(
        wid.HTML(
            f"<h1>Processing image {image_title} with {canny_impl_dropdown.value}</h1>"
        )
    )
    with LogTimer("Displaying input images"):
        display(
            wid.HBox(
                [
                    to_ipy_image(image_color, longest_side=image_size, upscale=True),
                    to_ipy_image(image_gray, longest_side=image_size, upscale=True),
                ]
            )
        )

    # GAUSSIAN BLURRING
    KEY_SIGMA_SLIDER = "sigma_slider"
    sigma_slider = wid.FloatSlider(
        value=memoize.get(KEY_SIGMA_SLIDER, default=3.0),
        min=0.1,
        max=20.0,
        step=0.1,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".1f",
        description="Sigma",
    )
    KEY_DETAIL_PLOT_GAUSS_CHECKBOX = "detail_plot_gauss_checkbox"
    detail_plot_gauss_checkbox = wid.Checkbox(
        value=memoize.get(KEY_DETAIL_PLOT_GAUSS_CHECKBOX, default=False),
        description="Show Gaussian Blur Details",
        **widgets_styling,
    )
    default_gauss_button = wid.Button(
        description="Default Values",
        **widgets_styling,
    )
    output_gaussian_blur = wid.Output()

    @output_gaussian_blur.capture(clear_output=True, wait=True)
    def on_gauss_change(config=None):
        memoize.set(KEY_SIGMA_SLIDER, sigma_slider.value)
        memoize.set(KEY_DETAIL_PLOT_GAUSS_CHECKBOX, detail_plot_gauss_checkbox.value)

        with LogTimer("Calculating gaussian blur"):
            image_blurred = canny_impl.blur_gauss(image_gray, sigma_slider.value)
        with LogTimer("Displaying blurred image"):
            display(to_ipy_image(image_blurred, longest_side=image_size, upscale=True))

        detail_plot_gauss = detail_plot_gauss_checkbox.value
        if detail_plot_gauss:
            with LogTimer("Displaying details of gaussian blur"):
                global plot_gauss_smart_fig
                impl_has_gaussian_kernel = hasattr(canny_impl, "gaussian_kernel")
                plot_gauss_smart_fig = SmartFigure(
                    figsize=(
                        fig_width,
                        fig_width / 2 * (1 + impl_has_gaussian_kernel) + 1,
                    ),
                )
                fig_gauss = plot_gauss_smart_fig.get_fig()
                ax = (
                    fig_gauss.add_subplot(1 + impl_has_gaussian_kernel, 2, 1),
                    fig_gauss.add_subplot(1 + impl_has_gaussian_kernel, 2, 2),
                )
                plot_matrix(ax[0], image_gray, title="Original Image")
                plot_matrix(ax[1], image_blurred, title="Blurred Image")

                if impl_has_gaussian_kernel:
                    kernel = canny_impl.gaussian_kernel(sigma_slider.value)
                    ax_3d = fig_gauss.add_subplot(2, 2, 3, projection="3d")
                    plot_kernel(
                        ax_3d,
                        kernel,
                        title=f"Gaussian Kernel (σ={np.round(sigma_slider.value, 1)})",
                    )

                fig_gauss.tight_layout()
                fig_gauss.canvas.layout.min_width = "400px"
                fig_gauss.canvas.layout.flex = "1 1 auto"
                fig_gauss.canvas.layout.width = "auto"
                display(fig_gauss.canvas)

        # SOBEL GRADIENTS
        display(wid.HTML("<h2>Sobel Gradients</h2>"))

        with LogTimer("Calculating sobel gradients"):
            grad_mag, grad_dir = canny_impl.sobel_gradients(image_blurred)

        grad_mag = grad_mag.astype(np.float32)
        grad_dir = grad_dir.astype(np.float32)

        with LogTimer("Displaying sobel gradients"):
            grad_dir_color = cv2.applyColorMap(
                np.uint8((grad_dir + np.pi) / (2 * np.pi) * 255),
                cv2.COLORMAP_RAINBOW,
            )
            grad_dir_color = grad_dir_color.astype(np.float32) / 255.0
            image_gradients = np.append(
                cv2.cvtColor(grad_mag, cv2.COLOR_GRAY2BGR), grad_dir_color, axis=1
            )
            display(
                to_ipy_image(image_gradients, longest_side=image_size, upscale=True)
            )

        # NON-MAXIMUM SUPPRESSION
        KEY_DETAIL_PLOT_NON_MAX = "plot_non_max"
        detail_plot_non_max_checkbox = wid.Checkbox(
            value=memoize.get(KEY_DETAIL_PLOT_NON_MAX, default=False),
            description="Show Non-Maximum Suppression Details",
            **widgets_styling,
        )
        output_non_max = wid.Output()

        @output_non_max.capture(clear_output=True, wait=True)
        def on_non_max_change(config=None):
            memoize.set(KEY_DETAIL_PLOT_NON_MAX, detail_plot_non_max_checkbox.value)

            with LogTimer("Calculating Non-maximum suppression"):
                image_nms = canny_impl.non_max(grad_mag, grad_dir)

            detail_plot_non_max = detail_plot_non_max_checkbox.value
            if detail_plot_non_max:
                global plot_non_max_smart_fig
                plot_non_max_smart_fig = SmartFigure(
                    figsize=(
                        fig_width,
                        fig_width / 2 + 1,
                    ),
                )
                fig_non_max = plot_non_max_smart_fig.get_fig()
                ax = fig_non_max.add_subplot(1, 2, 1), fig_non_max.add_subplot(1, 2, 2)
                plot_matrix(ax[0], grad_mag, title="Gradient Magnitude")
                plot_matrix(ax[1], image_nms, title="Non-Maximum Suppression")

                fig_non_max.tight_layout()
                fig_non_max.canvas.layout.min_width = "400px"
                fig_non_max.canvas.layout.flex = "1 1 auto"
                fig_non_max.canvas.layout.width = "auto"
                display(fig_non_max.canvas)
            # with LogTimer("Displaying non-maximum suppression"):
            #    display(to_ipy_image(image_nms, longest_side=image_size, upscale=True))

            # HYSTERESIS AUTO THRESHOLDING
            KEY_DO_AUTO_THRESHOLDING = "do_auto_thresholding"
            do_auto_thresholding_checkbox = wid.Checkbox(
                value=memoize.get(KEY_DO_AUTO_THRESHOLDING, default=True),
                description="Auto Thresholding",
                **widgets_styling,
            )
            KEY_DETAIL_PLOT_HYST = "detail_plot_hyst"
            detail_plot_hyst_checkbox = wid.Checkbox(
                value=memoize.get(KEY_DETAIL_PLOT_HYST, default=False),
                description="Show Hysteresis Thresholding Details",
                **widgets_styling,
            )
            KEY_LOW_PROPORTION = "low_proportion"
            low_proportion_slider = wid.FloatSlider(
                value=memoize.get(KEY_LOW_PROPORTION, default=0.7),
                min=0.0,
                max=1.0,
                step=0.01,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format=".2f",
            )
            KEY_HIGH_PROPORTION = "high_proportion"
            high_proportion_slider = wid.FloatSlider(
                value=memoize.get(KEY_HIGH_PROPORTION, default=0.3),
                min=0.0,
                max=1.0,
                step=0.01,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format=".2f",
            )
            low_high_prop_slider_layout = wid.HBox(
                [
                    wid.VBox(
                        [
                            wid.HTML("Low Proportion"),
                            wid.HTML("High Proportion"),
                        ],
                    ),
                    wid.VBox([low_proportion_slider, high_proportion_slider]),
                ]
            )
            KEY_LOW_THRESHOLD = "low_threshold"
            low_slider = wid.FloatSlider(
                value=memoize.get(KEY_LOW_THRESHOLD, default=0.7),
                min=np.finfo(np.float32).eps,
                max=1.0,
                step=0.01,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format=".2f",
            )
            KEY_HIGH_THRESHOLD = "high_threshold"
            high_slider = wid.FloatSlider(
                value=memoize.get(KEY_HIGH_THRESHOLD, default=0.3),
                min=0.0,
                max=1.0,
                step=0.01,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format=".2f",
            )
            low_high_slider_layout = wid.HBox(
                [
                    wid.VBox([wid.HTML("Low Threshold"), wid.HTML("High Threshold")]),
                    wid.VBox([low_slider, high_slider]),
                ]
            )
            hyst_sliders = [
                low_slider,
                high_slider,
                low_proportion_slider,
                high_proportion_slider,
            ]
            default_hyst_button = wid.Button(
                description="Default Values",
                **widgets_styling,
            )
            output_hysteresis = wid.Output()

            @output_hysteresis.capture(clear_output=True, wait=True)
            def on_auto_thresholding_change(config=None):
                memoize.set(
                    KEY_DO_AUTO_THRESHOLDING, do_auto_thresholding_checkbox.value
                )
                memoize.set(KEY_LOW_PROPORTION, low_proportion_slider.value)
                memoize.set(KEY_HIGH_PROPORTION, high_proportion_slider.value)
                memoize.set(KEY_LOW_THRESHOLD, low_slider.value)
                memoize.set(KEY_HIGH_THRESHOLD, high_slider.value)

                for slider in hyst_sliders:
                    slider.unobserve_all()

                def calculate_auto_thresholds(low_proportion, high_proportion):
                    low_high_prop = np.array(
                        [low_proportion, high_proportion], dtype=np.float32
                    )
                    low_high = canny_impl.compute_hysteresis_auto_thresholds(
                        image_nms, low_high_prop
                    )
                    return low_high[0], low_high[1]

                do_auto_thresholding = do_auto_thresholding_checkbox.value
                if do_auto_thresholding:
                    low_proportion_slider.observe(
                        on_auto_thresholding_change, names="value"
                    )
                    high_proportion_slider.observe(
                        on_auto_thresholding_change, names="value"
                    )

                    display(low_high_prop_slider_layout)

                    with LogTimer("Calculating auto thresholds for hysteresis"):
                        low, high = calculate_auto_thresholds(
                            low_proportion_slider.value, high_proportion_slider.value
                        )

                    if hasattr(canny_impl, "HISTOGRAM_BIN_COUNT"):
                        histogram_bin_count = canny_impl.HISTOGRAM_BIN_COUNT
                        bin_size = 1.0 / histogram_bin_count
                        low_bin = int(low / bin_size)
                        high_bin = int(high / bin_size)

                        threshold_message = (
                            f"Low threshold: {low:.2f} ({low_bin}), "
                            f"High threshold: {high:.2f} ({high_bin})"
                        )
                    else:
                        threshold_message = (
                            f"Low threshold: {low:.2f}, High threshold: {high:.2f}"
                        )

                    logging.info(threshold_message)
                    display(
                        wid.HTML(
                            "<span style='font-size: 1.1em; font-weight: bold'>"
                            f"{threshold_message}"
                            "</span>"
                        )
                    )
                else:
                    low_slider.observe(on_auto_thresholding_change, names="value")
                    high_slider.observe(on_auto_thresholding_change, names="value")

                    def use_auto_threshold_values(config=None):
                        low, high = calculate_auto_thresholds(
                            low_proportion_slider.value, high_proportion_slider.value
                        )
                        low_slider.value = low
                        high_slider.value = high
                        on_auto_thresholding_change()

                    use_auto_threshold_values_button = wid.Button(
                        description="Use auto threshold values",
                        **widgets_styling,
                    )
                    use_auto_threshold_values_button.on_click(use_auto_threshold_values)

                    display(
                        wid.HBox(
                            [
                                wid.VBox(
                                    [
                                        wid.HTML("Manual Thresholds"),
                                        low_high_slider_layout,
                                    ]
                                ),
                                wid.VBox(
                                    [
                                        use_auto_threshold_values_button,
                                        low_high_prop_slider_layout,
                                    ]
                                ),
                            ]
                        )
                    )

                    low = low_slider.value
                    high = high_slider.value

                low_high = np.array([low, high], dtype=np.float32)

                # HYSTERESIS

                debug_hyst_cuda_shared_mem_copy = False
                if (
                    debug_hyst_cuda_shared_mem_copy
                    and image_nms.shape[0] == image_nms.shape[1]
                ):
                    # DEBUG CUDA
                    image_nms[image_nms <= np.finfo(np.float32).eps] = 0.1

                    # top left
                    image_nms[0, 0] = 0.6
                    image_nms[2, 0] = 0.4
                    image_nms[0, 2] = 0.5
                    # bottom left
                    image_nms[-1, 0] = 0.6
                    image_nms[-3, 0] = 0.4
                    image_nms[-1, 2] = 0.5
                    # top right
                    image_nms[0, -1] = 0.6
                    image_nms[2, -1] = 0.4
                    image_nms[0, -3] = 0.5
                    # bottom right
                    image_nms[-1, -1] = 0.6
                    image_nms[-3, -1] = 0.4
                    image_nms[-1, -3] = 0.5

                    middle = image_nms.shape[0] // 2
                    # middle middle
                    image_nms[middle, middle] = 0.6
                    image_nms[middle + 2, middle] = 0.4
                    image_nms[middle, middle + 2] = 0.5
                    image_nms[middle - 2, middle] = 0.4
                    image_nms[middle, middle - 2] = 0.5
                    image_nms[middle + 1, middle + 1] = 0.3
                    image_nms[middle - 1, middle - 1] = 0.3
                    image_nms[middle + 1, middle - 1] = 0.3
                    image_nms[middle - 1, middle + 1] = 0.3
                    image_nms[middle + 1, middle] = 0.2
                    image_nms[middle - 1, middle] = 0.2
                    image_nms[middle, middle + 1] = 0.2
                    image_nms[middle, middle - 1] = 0.2
                    # top middle
                    image_nms[0, middle] = 0.6
                    image_nms[2, middle] = 0.4
                    image_nms[0, middle + 2] = 0.5
                    image_nms[0, middle - 2] = 0.5
                    # bottom middle
                    image_nms[-1, middle] = 0.6
                    image_nms[-3, middle] = 0.4
                    image_nms[-1, middle + 2] = 0.5
                    image_nms[-1, middle - 2] = 0.5
                    # left middle
                    image_nms[middle, 0] = 0.6
                    image_nms[middle, 2] = 0.5
                    image_nms[middle + 2, 0] = 0.4
                    image_nms[middle - 2, 0] = 0.4
                    # right middle
                    image_nms[middle, -1] = 0.6
                    image_nms[middle, -3] = 0.5
                    image_nms[middle + 2, -1] = 0.4
                    image_nms[middle - 2, -1] = 0.4

                    image_nms[image_nms == 1.0] = 0.9
                    image_nms[28, 16] = 1.0
                    image_nms[30, 30] = 0.8

                with LogTimer("Calculating hysteresis thresholding"):
                    image_edges = canny_impl.hysteresis(image_nms, low_high)

                detail_plot_hyst = detail_plot_hyst_checkbox.value
                if detail_plot_hyst:
                    global plot_hysteresis_smart_fig
                    plot_hysteresis_smart_fig = SmartFigure(
                        figsize=(
                            fig_width,
                            fig_width / 2 + 1,
                        ),
                    )
                    fig_hysteresis = plot_hysteresis_smart_fig.get_fig()
                    ax = (
                        fig_hysteresis.add_subplot(1, 2, 1),
                        fig_hysteresis.add_subplot(1, 2, 2),
                    )
                    plot_matrix(ax[0], image_nms, title="Non-Maximum Suppression")
                    plot_matrix(ax[1], image_edges, title="Edges")

                    fig_hysteresis.tight_layout()
                    fig_hysteresis.canvas.layout.min_width = "400px"
                    fig_hysteresis.canvas.layout.flex = "1 1 auto"
                    fig_hysteresis.canvas.layout.width = "auto"
                    display(fig_hysteresis.canvas)

                image_overlay = np.copy(image_color)
                image_overlay //= 4
                image_overlay[image_edges != 0] = [255, 0, 255]
                with LogTimer("Displaying edges"):
                    display(
                        wid.HBox(
                            [
                                to_ipy_image(
                                    image_edges, longest_side=image_size, upscale=True
                                ),
                                to_ipy_image(
                                    image_overlay, longest_side=image_size, upscale=True
                                ),
                            ]
                        )
                    )

            do_auto_thresholding_checkbox.observe(
                on_auto_thresholding_change, names="value"
            )
            detail_plot_hyst_checkbox.observe(
                on_auto_thresholding_change, names="value"
            )

            def default_hyst(config=None):
                memoize.delete_keys(
                    [
                        KEY_DO_AUTO_THRESHOLDING,
                        KEY_DETAIL_PLOT_HYST,
                        KEY_LOW_PROPORTION,
                        KEY_HIGH_PROPORTION,
                        KEY_LOW_THRESHOLD,
                        KEY_HIGH_THRESHOLD,
                    ]
                )
                on_gauss_change()

            default_hyst_button.on_click(default_hyst)

            display(
                wid.VBox(
                    [
                        wid.HTML("<h2>Hysteresis Thresholding</h2>"),
                        wid.HBox(
                            [
                                do_auto_thresholding_checkbox,
                                detail_plot_hyst_checkbox,
                                default_hyst_button,
                            ]
                        ),
                        output_hysteresis,
                    ]
                )
            )
            on_auto_thresholding_change()

        detail_plot_non_max_checkbox.observe(on_non_max_change, names="value")

        display(
            wid.VBox(
                [
                    wid.HTML("<h2>Non-Maximum Suppression</h2>"),
                    wid.HBox([detail_plot_non_max_checkbox]),
                    output_non_max,
                ]
            )
        )
        on_non_max_change()

    sigma_slider.observe(on_gauss_change, names="value")
    detail_plot_gauss_checkbox.observe(on_gauss_change, names="value")

    def default_gauss(config=None):
        memoize.delete_keys([KEY_SIGMA_SLIDER, KEY_DETAIL_PLOT_GAUSS_CHECKBOX])
        on_menu_change()

    default_gauss_button.on_click(default_gauss)

    display(
        wid.VBox(
            [
                wid.HTML("<h2>Gaussian Blur</h2>"),
                wid.HBox(
                    [sigma_slider, detail_plot_gauss_checkbox, default_gauss_button]
                ),
                output_gaussian_blur,
            ]
        )
    )
    on_gauss_change()


image_dropdown.observe(on_menu_change, names="value")
canny_impl_dropdown.observe(on_menu_change, names="value")
refresh_button.on_click(on_menu_change)

display(
    wid.VBox([wid.HBox([image_dropdown, canny_impl_dropdown, refresh_button]), output])
)
on_menu_change()
